In [2]:
import os
import keras
from keras.preprocessing import image

In [14]:
latent_dim = 32
height = 32
width = 32
channels = 3
iterations = 10000
batch_size = 20
save_dir = 'save_img/'

In [5]:
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

x_train = x_train[y_train.flatten() == 6]
x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

## Generator

In [6]:
from keras import layers
import numpy as np

In [7]:
generator_input = keras.Input(shape=(latent_dim,))

layer_1 = layers.Dense(128*16*16)(generator_input)
activation_1 = layers.LeakyReLU()(layer_1)
layer_reshape = layers.Reshape((16, 16, 128))(activation_1)

layer_2 = layers.Conv2D(256, 5, padding='same')(layer_reshape)
activation_2 = layers.LeakyReLU()(layer_2)

layer_3 = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(activation_2)
activation_3 = layers.LeakyReLU()(layer_3)


layer_4 = layers.Conv2D(256, 5, padding='same')(activation_3)
activation_4 = layers.LeakyReLU()(layer_4)

layer_5 = layers.Conv2D(256, 5, padding='same')(activation_4)
activation_5 = layers.LeakyReLU()(layer_5)

layer_6 = layers.Conv2D(channels, 7, activation='tanh', padding='same')(activation_5)
generator = keras.models.Model(generator_input, layer_6)
generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048

## Discriminator 

In [9]:
discriminator_input = layers.Input(shape=(height, width, channels))

dis_1 = layers.Conv2D(128, 3)(discriminator_input)
dis_act1 = layers.LeakyReLU()(dis_1)
dis_2 = layers.Conv2D(128, 4, strides=2)(dis_act1)
dis_act2 = layers.LeakyReLU()(dis_2)
dis_3 = layers.Conv2D(128, 4, strides=2)(dis_act2)
dis_act3 = layers.LeakyReLU()(dis_3)
dis_4 = layers.Conv2D(128, 4, strides=2)(dis_act3)
dis_act4 = layers.LeakyReLU()(dis_4)

dense_1 = layers.Flatten()(dis_act4)
dropout = layers.Dropout(0.4)(dense_1)
output = layers.Dense(1, activation='sigmoid')(dropout)

discriminator = keras.models.Model(discriminator_input, output)
discriminator.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 6, 6, 128)         0   

In [10]:
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

## DCGAN

In [11]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

In [12]:
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [18]:
start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)),np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_images, labels)
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors,misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)
    
        #img = image.array_to_img(generated_images[0] * 255., scale=False)
        #img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))
        img.save("generated_img_{i}.png".format(i=step))
        #img = image.array_to_img(real_images[0] * 255., scale=False)
        #img.save(os.path.join(save_dir,'real_frog' + str(step) + '.png'))

discriminator loss: 0.6843525
adversarial loss: 0.8974575
discriminator loss: 0.7330637
adversarial loss: 0.9340439
discriminator loss: 0.6613839
adversarial loss: 1.0980215
discriminator loss: 0.6642051
adversarial loss: 0.794799
discriminator loss: 0.7621855
adversarial loss: 0.76421946
discriminator loss: 0.68819726
adversarial loss: 0.790277
discriminator loss: 0.69307005
adversarial loss: 0.7817064
discriminator loss: 0.7024203
adversarial loss: 0.7608582
discriminator loss: 0.6932648
adversarial loss: 0.8609104
discriminator loss: 0.93626815
adversarial loss: 1.0827628
discriminator loss: 0.6859482
adversarial loss: 0.8159696
discriminator loss: 0.6976817
adversarial loss: 0.80272615
discriminator loss: 0.70260125
adversarial loss: 0.72936183
discriminator loss: 0.70487344
adversarial loss: 0.8326454
discriminator loss: 0.6974661
adversarial loss: 0.70359874
discriminator loss: 0.6959611
adversarial loss: 0.84808826
discriminator loss: 0.6739539
adversarial loss: 0.8162109
discri